In [2]:
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import fwi_aux as fwi_xxx
from datetime import timedelta
import joblib
import pickle

from keras.models import load_model
from matplotlib import pylab as plt
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense,LSTM, BatchNormalization, Dropout, Bidirectional, Flatten, LeakyReLU, Conv3D, MaxPooling3D

import locale
locale.setlocale(locale.LC_TIME, '')
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

path = os.getcwd() + '/'
path_out = 'RESULTADOS_MONTECARLO/'

In [3]:
nc_org     = xr.open_dataset('../redes/AL/DATA_DAY/2m_temperature.nc')-273.15
#nc_tem_no     = xr.open_dataset('DATA_DAY/t2m_mod.nc')-273.15
nc_tem     = xr.open_dataset('../redes/AL/DATA_TEM_MOD/t2m_mod.nc')-273.15
nc_rad     = xr.open_dataset('../redes/AL/DATA_DAY/surface_solar_radiation_downwars.nc')/3600
nc_soiltem = xr.open_dataset('../redes/AL/DATA_DAY/soil_temperature_level_1.nc')-273.15
nc_pre     = xr.open_dataset('../redes/AL/DATA_DAY/total_precipitation.nc')*1000
nc_dew     = xr.open_dataset('../redes/AL/DATA_DAY/2m_dewpoint_temperature.nc')-273.15
nc_uwind   = xr.open_dataset('../redes/AL/DATA_DAY/10m_u_component_of_wind.nc')
nc_vwind   = xr.open_dataset('../redes/AL/DATA_DAY/10m_v_component_of_wind.nc')
nc_leafhigh= xr.open_dataset('../redes/AL/DATA_DAY/leaf_area_index_high_vegetation.nc')
nc_leaflow = xr.open_dataset('../redes/AL/DATA_DAY/leaf_area_index_low_vegetation.nc')
nc_skin    = xr.open_dataset('../redes/AL/DATA_DAY/skin_reservoir_content.nc')
nc_vol     = xr.open_dataset('../redes/AL/DATA_DAY/volumetric_soil_water_layer_1.nc')
nc_eva     = xr.open_dataset('../redes/AL/DATA_DAY/evaporation.nc')*(-1000)


nc_BUI  = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/BUI.nc')
nc_DC   = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/DC.nc')
nc_DMC  = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC//DMC.nc')
nc_DSR  = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/DSR.nc')
nc_FFMC = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/FFMC.nc')
nc_FWI  = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/FWI.nc')
nc_ISI  = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/ISI.nc')
nc_M    = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/M.nc')
nc_WIND = xr.open_dataset('../FWI/RESULTADOS_FWI10_NC/WIND.nc')

nc_org = nc_org.rename({'t2m': 't2m_org'})

tem      = nc_tem.t2m.assign_coords(strftime = nc_tem.t2m.strftime.values.astype('datetime64[ns]'))  #DESACTIVAR con datos modificados en tiempo o long
pre      = nc_pre.tp.assign_coords(strftime = nc_pre.tp.strftime.values.astype('datetime64[ns]'))
rad      = nc_rad.ssrd.assign_coords(strftime = nc_rad.ssrd.strftime.values.astype('datetime64[ns]'))
soiltem  = nc_soiltem.stl1.assign_coords(strftime = nc_soiltem.stl1.strftime.values.astype('datetime64[ns]'))     
dew      = nc_dew.d2m.assign_coords(strftime = nc_dew.d2m.strftime.values.astype('datetime64[ns]'))    
uwind    = nc_uwind.u10.assign_coords(strftime = nc_uwind.u10.strftime.values.astype('datetime64[ns]'))   
vwind    = nc_vwind.v10.assign_coords(strftime = nc_vwind.v10.strftime.values.astype('datetime64[ns]'))  
leafhigh = nc_leafhigh.lai_hv.assign_coords(strftime = nc_leafhigh.lai_hv.strftime.values.astype('datetime64[ns]'))
leaflow  = nc_leaflow.lai_lv.assign_coords(strftime = nc_leaflow.lai_lv.strftime.values.astype('datetime64[ns]'))
skin     = nc_skin.src.assign_coords(strftime = nc_skin.src.strftime.values.astype('datetime64[ns]'))
vol      = nc_vol.swvl1.assign_coords(strftime = nc_vol.swvl1.strftime.values.astype('datetime64[ns]'))
eva      = nc_eva.e.assign_coords(strftime = nc_eva.e.strftime.values.astype('datetime64[ns]'))

nc_BUI  = nc_BUI.BUI.rename({'time':'strftime'}) 
nc_DC   = nc_DC.DC.rename({'time':'strftime'}) 
nc_DMC  = nc_DMC.DCM.rename({'time':'strftime'})
nc_DSR  = nc_DSR.DSR.rename({'time':'strftime'})
nc_FFMC = nc_FFMC.FFMC.rename({'time':'strftime'})
nc_FWI  = nc_FWI.FWI.rename({'time':'strftime'})
nc_ISI  = nc_ISI.ISI.rename({'time':'strftime'})
nc_M    = nc_M.M.rename({'time':'strftime'})
nc_WIND = nc_WIND.WIND.rename({'time':'strftime'})

BUI  = nc_BUI.assign_coords(strftime = nc_BUI.strftime.values.astype('datetime64[ns]'))  
DC   = nc_DC.assign_coords(strftime = nc_DC.strftime.values.astype('datetime64[ns]'))  
DMC  = nc_DMC.assign_coords(strftime = nc_DMC.strftime.values.astype('datetime64[ns]'))  
DSR  = nc_DSR.assign_coords(strftime = nc_DSR.strftime.values.astype('datetime64[ns]'))  
FFMC = nc_FFMC.assign_coords(strftime = nc_FFMC.strftime.values.astype('datetime64[ns]')) 
FWI  = nc_FWI.assign_coords(strftime = nc_FWI.strftime.values.astype('datetime64[ns]')) 
ISI  = nc_ISI.assign_coords(strftime = nc_ISI.strftime.values.astype('datetime64[ns]'))  
M    = nc_M.assign_coords(strftime = nc_M.strftime.values.astype('datetime64[ns]')) 
WIND = nc_WIND.assign_coords(strftime = nc_WIND.strftime.values.astype('datetime64[ns]'))


In [12]:
model = joblib.load("../Redes/AL/random_forest.joblib")   #correr con venv sml
#with open("../Redes/AL/random_forest_model.pkl", "rb") as file:
#    model = pickle.load(file)
#model = pickle.load("../Redes/AL/random_forest.pkl")   #correr con venv sml


#rf_load = loaded_rf.predict(x_test)

inicio_conteo = len(nc_tem.t2m[:int(len(nc_tem.t2m)*0.8)]) 
test_data_tem = nc_tem.t2m[inicio_conteo:,:,:].copy()
#test_data_tem = test_data_tem.rename({'time':'strftime'})

In [13]:
test_data_tem = nc_tem.t2m[inicio_conteo:,:,:].copy()

def nc4(datos_datos, name):

    from datetime import datetime, timedelta
    from netCDF4 import Dataset,num2date,date2num
        
    a = test_data_tem.strftime[0].values
    b = test_data_tem.strftime[len(test_data_tem.strftime)-1].values

    nyears = len(test_data_tem.strftime);
    unout = 'days since ' + str(pd.to_datetime(a).year) + '-' + str(pd.to_datetime(a).month) + '-' + str(pd.to_datetime(a).day)

    nx, ny = (len(test_data_tem.latitude), len(test_data_tem.longitude))

    lat = test_data_tem.latitude.values
    lon = test_data_tem.longitude.values

    dataout = datos_datos
        
    datesout = np.arange(datetime(int(pd.to_datetime(a).year),int(pd.to_datetime(a).month),int(pd.to_datetime(a).day)),
                        datetime(int((pd.to_datetime(b) + timedelta(days=1)).year),int((pd.to_datetime(b) + timedelta(days=1)).month),int((pd.to_datetime(b) + timedelta(days=1)).day)),
                        timedelta(days = 1)).astype(datetime)
        
    ncout = Dataset(path + path_out + name + '.nc','w','NETCDF3')
    ncout.createDimension('longitude',ny)
    ncout.createDimension('latitude',nx)
    ncout.createDimension('time',nyears)
    lonvar = ncout.createVariable('longitude','float32',('longitude'));lonvar[:] = lon
    latvar = ncout.createVariable('latitude','float32',('latitude'));latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=date2num(datesout,unout)
    myvar = ncout.createVariable(name,'float32',('time','latitude','longitude'));myvar.setncattr('units','');myvar[:] = dataout
    ncout.close()

In [14]:
def generador_ramdom2(name_distribution,nc, latitude, longitude,i):
    
    distribution = getattr(st, name_distribution)  
    diff         = nc[:,latitude,longitude].median() - nc[i,latitude,longitude].values  
    params       = distribution.fit(nc[:,latitude,longitude] - diff.to_numpy())        
    arg          = params[:-2]
    scale        = params[-1]
    loc          = params[-2]
    random       = distribution.rvs(size = num_monte, loc=loc, scale=scale, *arg)
    random[random > nc[:,latitude,longitude].max().values] = nc[:,latitude,longitude].max().values
    random[random < nc[:,latitude,longitude].min().values] = nc[:,latitude,longitude].min().values
    
    return(random)

def resultados_prob (prediccion, categoria, num_monte):      
    lista_names= []
    matriz_prob = np.zeros((1, len(T2.longitude) * len(T2.latitude)))
    suma = 0
    
    for latitude in np.arange(0,len(T2.latitude),1):
        for longitude in np.arange(0,len(T2.longitude),1):
            probabilidad        = (np.count_nonzero(prediccion[:,latitude, longitude] == categoria)/num_monte)*100
            matriz_prob[:,suma] = probabilidad; suma += 1
            lista_names         = np.append(lista_names, str(latitude) + ',' + str(longitude))
            
    df_prob        = pd.DataFrame(matriz_prob,columns = lista_names)  
    resultado_prob = df_prob.values.reshape(1,len(T2.latitude), len(T2.longitude))  
    return np.round(resultado_prob,1)


def data_model(matriz_var_random, lista_names, num_monte, T2):
    
    df_var_random     = pd.DataFrame(matriz_var_random,columns = lista_names)
    result_var_random = df_var_random.values.reshape(num_monte,len(T2.latitude), len(T2.longitude))
    dim_var_random    = np.reshape(result_var_random, (result_var_random.shape[0], result_var_random.shape[1],result_var_random.shape[2],1))
    return dim_var_random

def threshold(data,percentil): 
    matriz_thrs = np.zeros((1, len(T2.longitude) * len(T2.latitude)))
    suma_thrs   = 0
    lista_names_thrs = []

    for latitude in np.arange(0,len(data.latitude),1):

        for longitude in np.arange(0,len(data.longitude),1):

            data_pixel = data[:,latitude,longitude].copy()
            thrs       = np.round(np.percentile(data_pixel,percentil),4)

            matriz_thrs[:,suma_thrs] = thrs ; suma_thrs += 1
            lista_names_thrs         = np.append(lista_names_thrs, str(latitude) + ',' + str(longitude))

    df_thrs = pd.DataFrame(matriz_thrs,columns = lista_names_thrs)
    resultado_thrs  = df_thrs.values.reshape(1,len(data.latitude), len(data.longitude))
    
    
    return resultado_thrs

In [ ]:
num_monte = 100 # numero de iteraciones en el mismo dia
T2 = nc_org.t2m_org

thrs_alta  = 29.5
thrs_media = 24.71
clases_reales2 = nc_tem.t2m[:,:,:].values.copy()
clases_reales2[clases_reales2 < thrs_alta]  = 0 
clases_reales2[clases_reales2 >= thrs_media] = 3 #6


for ciclo in np.arange(inicio_conteo,len(nc_pre.tp),1): 

    print(ciclo)
    fin         = ciclo
    ini         = ciclo - 10
    suma        = 0
    lista_names = []
    
    matriz_prediccion     = np.zeros((num_monte, len(T2.longitude) * len(T2.latitude)))
 

    for latitude in np.arange(0,len(T2.latitude),1):
        for longitude in np.arange(0,len(T2.longitude),1):
        
            eva_random_1     = generador_ramdom2('burr', nc_eva.e, latitude, longitude, fin)
            v10_random_1     = generador_ramdom2('norm', nc_vwind.v10, latitude, longitude, fin)
            tp_random_1      = generador_ramdom2('expon', nc_pre.tp, latitude, longitude, fin) #genpareto, lognorm
            lai_hv_random_1  = generador_ramdom2('norm', nc_leafhigh.lai_hv, latitude, longitude, fin)
            src_random_1     = generador_ramdom2('norm', nc_skin.src, latitude, longitude, fin)
            ssrd_random_1    = generador_ramdom2('burr', nc_rad.ssrd, latitude, longitude, fin)
            t2m_org_random_1 = generador_ramdom2('burr', nc_org.t2m_org, latitude, longitude, fin)
            u10_random_1     = generador_ramdom2('norm', nc_uwind.u10, latitude, longitude, fin)
            d2m_random_1     = generador_ramdom2('burr', nc_dew.d2m, latitude, longitude, fin)  
            stl1_random_1    = generador_ramdom2('norm', nc_soiltem.stl1, latitude, longitude, fin)
            
            eva_random_1[eva_random_1 > nc_eva.e.max().values]                 = nc_eva.e.max() 
            v10_random_1[v10_random_1 > nc_vwind.v10.max().values]             = nc_vwind.v10.max()  
            tp_random_1[tp_random_1 > nc_pre.tp.max().values]                  = nc_pre.tp.max()  
            lai_hv_random_1[lai_hv_random_1 > nc_leafhigh.lai_hv.max().values] = nc_leafhigh.lai_hv.max()  
            src_random_1[src_random_1 > nc_skin.src.max().values]              = nc_skin.src.max()  
            ssrd_random_1[ssrd_random_1 > nc_rad.ssrd.max().values]            = nc_rad.ssrd.max()
            t2m_org_random_1[t2m_org_random_1 > nc_org.t2m_org.max().values]   = nc_org.t2m_org.max()
            u10_random_1[u10_random_1 > nc_uwind.u10.max().values]             = nc_uwind.u10.max()                  
            d2m_random_1[d2m_random_1 > nc_dew.d2m.max().values]               = nc_dew.d2m.max()
            stl1_random_1[stl1_random_1 > nc_soiltem.stl1.max().values]        = nc_soiltem.stl1.max()

            eva_random_1[eva_random_1 < nc_eva.e.min().values]                 = nc_eva.e.min()
            v10_random_1[v10_random_1 < nc_vwind.v10.min().values]             = nc_vwind.v10.min()
            tp_random_1[tp_random_1 < nc_pre.tp.min().values]                  = nc_pre.tp.min()
            lai_hv_random_1[lai_hv_random_1 < nc_leafhigh.lai_hv.min().values] = nc_leafhigh.lai_hv.min()
            src_random_1[src_random_1 < nc_skin.src.min().values]              = nc_skin.src.min()
            ssrd_random_1[ssrd_random_1 < nc_rad.ssrd.min().values]            = nc_rad.ssrd.min()
            t2m_org_random_1[t2m_org_random_1 < nc_org.t2m_org.min().values]   = nc_org.t2m_org.min()
            u10_random_1[u10_random_1 < nc_uwind.u10.min().values]             = nc_uwind.u10.min()
            d2m_random_1[d2m_random_1 < nc_dew.d2m.min().values]               = nc_dew.d2m.min()
            stl1_random_1[stl1_random_1 < nc_soiltem.stl1.min().values]        = nc_soiltem.stl1.min()
            
 
            mylist_DCM = []
            mylist_DSR = []
            mylist_WIND = []
             
            for ii in np.arange(0, num_monte, 1):  #Indices FWI

                d2m_fwi     = nc_dew.d2m[ini:fin+1,latitude,longitude].to_dataframe().drop(['longitude', 'latitude'], axis=1)  
                t2m_org_fwi = nc_org.t2m_org[ini:fin+1,latitude,longitude].to_dataframe().drop(['longitude', 'latitude'], axis=1)
                v10_fwi     = nc_vwind.v10[ini:fin+1,latitude,longitude].to_dataframe().drop(['longitude', 'latitude'], axis=1)
                u10_fwi     = nc_uwind.u10[ini:fin+1,latitude,longitude].to_dataframe().drop(['longitude', 'latitude'], axis=1)
                tp_fwi      = nc_pre.tp[ini:fin+1,latitude,longitude].to_dataframe().drop(['longitude', 'latitude'], axis=1)
            
                dataframe_unido = pd.DataFrame([])
                lista_nc        = [d2m_fwi, t2m_org_fwi, v10_fwi, u10_fwi, tp_fwi]
                
                for i in np.arange(0,len(lista_nc),1):
                    array_org       = lista_nc[i][list(lista_nc[i].keys())[0]].values
                    dataframe_unico = pd.DataFrame(array_org, columns = list(lista_nc[i].keys()))
                    dataframe_unido = pd.concat([dataframe_unido,dataframe_unico],axis = 1)
                    
                fechas                    = pd.DataFrame(nc_pre.tp[ini:fin+1,latitude,longitude].strftime.values, columns = ['Fecha'])
                dataframe_unido           = pd.concat([fechas,dataframe_unido],axis = 1)
                dataframe_unido           = dataframe_unido.set_index('Fecha')
                fechas                    = pd.DataFrame(dataframe_unido.index, columns = ['Fecha'])
                dataframe_unido.index     = pd.to_datetime(dataframe_unido.index)
                dataframe_unido.iloc[-1:] = [d2m_random_1[ii],t2m_org_random_1[ii],v10_random_1[ii], u10_random_1[ii], tp_random_1[ii]]

                dataframe_unido['rh']   = 100*(np.exp((17.625*dataframe_unido['d2m'])/(243.04+dataframe_unido['d2m']))/np.exp((17.625*dataframe_unido['t2m_org'])/(243.04+dataframe_unido['t2m_org'])))
                dataframe_unido['WIND'] = np.sqrt(dataframe_unido['u10']**2 + dataframe_unido['v10']**2) * 3.6

                dataframe_unido['MES'] = dataframe_unido.index.strftime('%m')
                dataframe_unido        = dataframe_unido.reset_index()[['MES','t2m_org','rh', 'WIND', 'tp']].rename({'t2m_org':'TEMP','rh':'RH', 'tp':'RAIN'}, axis = 1)
                
                data_csv = fwi_xxx.xxx(dataframe_unido)
                data_csv = data_csv.iloc[10]
                
                mylist_DCM.append(data_csv['DCM']) 
                mylist_DSR.append(data_csv['DSR'])
                mylist_WIND.append(data_csv['WIND'])
                    
            DCM_random_1 = np.array(mylist_DCM)   
            DSR_random_1 = np.array(mylist_DSR)
            WIND_random_1 = np.array(mylist_WIND)
            
            DCM_random_1[DCM_random_1    > DCM_random_1.max()]  = DCM_random_1.max()
            DSR_random_1[DSR_random_1    > DSR_random_1.max()]  = DSR_random_1.max()
            WIND_random_1[WIND_random_1  > WIND_random_1.max()] = WIND_random_1.max()
            
            
            DCM_random_1[DCM_random_1    < DCM_random_1.min()]  = DCM_random_1.min()
            DSR_random_1[DSR_random_1    < DSR_random_1.min()]  = DSR_random_1.min()
            WIND_random_1[WIND_random_1  < WIND_random_1.min()] = WIND_random_1.min()
            
            x_test = np.stack((tp_random_1,eva_random_1,ssrd_random_1,stl1_random_1,lai_hv_random_1,src_random_1,DCM_random_1,DSR_random_1,WIND_random_1),axis = 1)
            x_test = pd.DataFrame(x_test, columns=['tp', 'e', 'ssrd','stl1','lai_hv','src','DCM','DSR','WIND'])
            prediccion_dia = model.predict(x_test)
            
            matriz_prediccion[:,suma]    = prediccion_dia;   suma += 1
            lista_names    = np.append(lista_names, str(latitude) + ',' + str(longitude))
            
            
    df_prediccion  = pd.DataFrame(matriz_prediccion,columns = lista_names).copy()
    prediccion = df_prediccion.values.reshape(100,7,7).copy()
                        
    prediccion[prediccion < thrs_media]  = 0    
    prediccion[prediccion >= thrs_alta]  = 4  
    prediccion[prediccion > thrs_media]  = 2 
    
    probabilidad_0 = resultados_prob(prediccion, 0, num_monte) # prediccion, categoria, num_monte
    probabilidad_2 = resultados_prob(prediccion, 2, num_monte)
    probabilidad_4 = resultados_prob(prediccion, 4, num_monte)  
    
    if ciclo == inicio_conteo:
        resultado_final_0 = probabilidad_0.copy()
        resultado_final_2 = probabilidad_2.copy()
        resultado_final_4 = probabilidad_4.copy()    
        
    else:
        resultado_final_0 = np.concatenate((resultado_final_0, probabilidad_0), axis=0) 
        resultado_final_2 = np.concatenate((resultado_final_2, probabilidad_2), axis=0)
        resultado_final_4 = np.concatenate((resultado_final_4, probabilidad_4), axis=0)
        

In [13]:
#nc4(resultado_final_0, 'class_RF_0')
#nc4(resultado_final_2, 'class_RF_2')
#nc4(resultado_final_4, 'class_RF_4')